In [ ]:

%load_ext autoreload
%autoreload 2
import torch
from torch.utils.data import DataLoader
import sys

sys.path.append("..")

from src.dvip import DVIP_Base
from src.likelihood import Gaussian
from src.generative_functions import SimplerBayesLinear
from src.layers_init import init_layers
from utils.dataset import Boston_Dataset
from utils.metrics import MetricsRegression
from utils.pytorch_learning import fit, fit_with_metrics, score


In [ ]:
params = {"vip_layers": [1],
        "genf": "BNN",
        "regression_coeffs": 20,
        "bnn_structure": [10, 10],
        "bnn_layer": SimplerBayesLinear,
        "activation": torch.tanh,
        "device": "cpu",
        "dtype": torch.float64,
        "seed": 2147483647,
        "fix_prior_noise": True,
        "genf_full_output": False,
        "final_layer_mu": 0,
        "inner_layers_mu": 0,
        "final_layer_sqrt": 1,
        "inner_layers_sqrt": 1e-5,
        "inner_layers_noise": -5,
        "final_layer_noise": None,
        "dropout": 0.0,
        "prior_kl": False,
        "zero_mean_prior": False,
        "input_prop": True,
}

In [ ]:

torch.manual_seed(params["seed"]);

In [ ]:
dataset = Boston_Dataset()
split = 0
train_dataset, train_test_dataset, test_dataset = dataset.get_split(test_size = 0.1, 
                                                                    seed = params["seed"] + split)

In [ ]:
batch_size = 100
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle = True)
train_test_loader = DataLoader(train_test_dataset, batch_size=batch_size)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

In [ ]:
layers = init_layers(train_dataset.inputs, dataset.output_dim, **params)


In [ ]:
likelihood = Gaussian(dtype = params["dtype"], device = params["device"])

In [ ]:
dvip = DVIP_Base(
    likelihood,
    layers,
    len(train_dataset),
    bb_alpha=0,
    num_samples=1,
    y_mean=train_dataset.targets_mean,
    y_std=train_dataset.targets_std,
    dtype=params["dtype"],
    device=params["device"]
)

In [ ]:
# Define optimizer and compile model
opt = torch.optim.Adam(dvip.parameters(), lr=0.001)


fit(
    dvip,
    train_loader,
    opt,
    epochs=30000,
    device=params["device"],
)


In [ ]:
metrics = MetricsRegression

In [ ]:
# Set the number of test samples to generate
dvip.num_samples = 200

# Test the model
test_metrics = score(dvip, test_loader, metrics, device=params["device"])

print("TEST RESULTS: ")
for k, v in test_metrics.items():
    print("\t - {}: {}".format(k, v))